## Requirements

### Imports

In [22]:
import os
import torch
from pathlib import Path

import random
import torch
import torchcam
import matplotlib.pyplot as plt
import numpy as np
import cv2

from torchcam.methods import GradCAM
from torchvision import transforms

In [23]:
import training

In [24]:
torch.manual_seed(42)

### Paths

In [25]:
SRC_DIR = Path.cwd()
ROOT_DIR = SRC_DIR.parent

DATA_DIR = os.path.join(ROOT_DIR, 'dataset')
PREPROCESSED_DIR = os.path.join(DATA_DIR, 'preprocessed')
CSV_PATH = os.path.join(DATA_DIR, 'csv_mappings', 'train.csv')

MODEL_DIR = os.path.join(ROOT_DIR, 'models')
BASELINE_DIR = os.path.join(MODEL_DIR, 'baselines')
BASELINE_FINETUNED_DIR = os.path.join(MODEL_DIR, 'baselines_finetuned')
RESULT_DIR = os.path.join(BASELINE_DIR, 'results')

### Load Data

In [26]:
BATCH_SIZE = 32

In [27]:
CLASS_NAMES = {
    0: "amanita",
    1: "boletus",
    2: "chantelle",
    3: "deterrimus",
    4: "rufus",
    5: "torminosus",
    6: "aurantiacum",
    7: "procera",
    8: "involutus",
    9: "russula"
}

In [28]:
train_loader, val_loader, test_loader = training.get_data_loaders(PREPROCESSED_DIR, CSV_PATH, BATCH_SIZE)

##### Set model to explain

In [29]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_CLASSES = 10 

In [30]:
model_type = 'resnet'  
# alexnet # resnet # vgg16 # densenet # efficientnet
# custom_alexnet custom_resnet

In [ ]:
model = training.load_model_for_explaining(model_type, NUM_CLASSES, DEVICE, finetuned=True)

c:\Users\ilian\Documents\Projects\git_projects\university\mushroom_classification\src\training.py:384: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(

### Explain

In [11]:
random_index = random.randint(0, len(test_loader.dataset) - 1)
image, label = test_loader.dataset[random_index]
image_tensor = image.to(DEVICE)

c:\Users\ilian\Documents\Projects\git_projects\university\mushroom_classification\src\training.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  image = torch.load(image_

In [ ]:
# Step 1: Initialize Grad-CAM
gradcam = GradCAM(model, target_layers=[model.layer4])  # Replace with your model's layer if necessary

# Step 2: Preprocess the image for model input (Ensure it's already a tensor with required transformations)
# Since the image is already a tensor, we skip transformations. If it's in PIL format, you'd apply the transform.
image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension

# Step 3: Perform Grad-CAM and get the heatmap
grayscale_cam = gradcam(image_tensor)
grayscale_cam = grayscale_cam[0, :]  # Get the first image in batch

# Step 4: Convert the grayscale heatmap to a color heatmap
heatmap = cv2.applyColorMap(np.uint8(255 * grayscale_cam), cv2.COLORMAP_JET)
heatmap = np.float32(heatmap) / 255

# Step 5: Get the original image
original_image = image.permute(1, 2, 0).cpu().numpy()  # Convert to HWC format

# Step 6: Overlay the heatmap on the original image
superimposed_img = heatmap + np.float32(original_image)
superimposed_img = superimposed_img / np.max(superimposed_img)  # Normalize to [0, 1]

# Step 7: Display the original image with the Grad-CAM heatmap overlay
plt.figure(figsize=(10, 10))
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()
